In [1]:
# read in data
import pandas as pd
data = pd.read_csv("clean_fulldata.csv")
data.head()

,Unnamed: 0,subreddit,subreddit_subscribers,title,id,author,created_utc,num_comments,score,selftext,url,upvote_ratio,comments,clean_title,clean_selftext,clean_comments
0,2,askSingapore,44675,Anyone know where to get tested as an Adult fo...,ng6k8b,summerfellxx,2021-05-19,6,8,I know mostly these tests are for children. Bu...,https://www.reddit.com/r/askSingapore/comments...,0.84,"[""is this recent or has it been going on since...",anyone know get tested adult auditory processi...,know mostly test child really need know wrong ...,is recent going since childhood nnif recent mi...
1,4,NationalServiceSG,11091,Help for adjustment disorder,mfnwal,ElijahThor00,2021-03-29,6,5,People say half the war is won when you report...,https://www.reddit.com/r/NationalServiceSG/com...,0.86,['pretty long unfortunately. :( i’m currently ...,help adjustment disorder,people say half war report mental illness long...,pretty long unfortunately currently going coun...
2,5,askSingapore,44675,Where to get assessed for eating disorders?,l0ac4u,kanicroquette,2021-01-19,9,42,"I might have an eating disorder, I don’t know ...",https://www.reddit.com/r/askSingapore/comments...,1.00,"[""Don't self diagnose. Cheapest way: got to po...",get assessed eating disorder,might eating disorder know want self diagnose ...,do self diagnose cheapest way got polyclinic s...
3,7,SGExams,77716,[RANT] Mental disorders,kzrgon,Vivid-Wing-3460,2021-01-18,7,44,After o's I have this long period of time to s...,https://www.reddit.com/r/SGExams/comments/kzrg...,0.95,"[""You sound exactly like me. What I can tell y...",rant mental disorder,ofs long period time sit really reflect unsolv...,you sound exactly like me tell thought simply ...
4,8,NationalServiceSG,11091,Downpes from adjustment disorder,kwbxjs,chin_chin_daisuki,2021-01-13,8,21,"Facing severe adjustment problems, had a menta...",https://www.reddit.com/r/NationalServiceSG/com...,0.90,"['upvote for chin_chin_daisuki', 'It can take ...",downpes adjustment disorder,facing severe adjustment problem mental breakd...,upvote chinchindaisuki it take month fast mont...


In [2]:
# filter for required columns
output_data = data[["created_utc", "author", "score", "num_comments", "clean_selftext", "selftext"]]
output_data.head()

,created_utc,author,score,num_comments,clean_selftext,selftext
0,2021-05-19,summerfellxx,8,6,know mostly test child really need know wrong ...,I know mostly these tests are for children. Bu...
1,2021-03-29,ElijahThor00,5,6,people say half war report mental illness long...,People say half the war is won when you report...
2,2021-01-19,kanicroquette,42,9,might eating disorder know want self diagnose ...,"I might have an eating disorder, I don’t know ..."
3,2021-01-18,Vivid-Wing-3460,44,7,ofs long period time sit really reflect unsolv...,After o's I have this long period of time to s...
4,2021-01-13,chin_chin_daisuki,21,8,facing severe adjustment problem mental breakd...,"Facing severe adjustment problems, had a menta..."


In [3]:
# add year and month columns
year = []
month = []
for rowIndex, row in output_data.iterrows():
    date = row["created_utc"].split("-")
    year.append(date[0])
    month.append(date[1])
output_data["year"] = year
output_data["month"] = month
output_data.head()

C:\Users\20jam\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,created_utc,author,score,num_comments,clean_selftext,selftext,year,month
0,2021-05-19,summerfellxx,8,6,know mostly test child really need know wrong ...,I know mostly these tests are for children. Bu...,2021,05
1,2021-03-29,ElijahThor00,5,6,people say half war report mental illness long...,People say half the war is won when you report...,2021,03
2,2021-01-19,kanicroquette,42,9,might eating disorder know want self diagnose ...,"I might have an eating disorder, I don’t know ...",2021,01
3,2021-01-18,Vivid-Wing-3460,44,7,ofs long period time sit really reflect unsolv...,After o's I have this long period of time to s...,2021,01
4,2021-01-13,chin_chin_daisuki,21,8,facing severe adjustment problem mental breakd...,"Facing severe adjustment problems, had a menta...",2021,01


In [4]:
# add contain_covid, contain_sg, contain_mental_health and relevancy columns
import re
contain_covid = []
contain_sg = []
contain_mental_health = []
relevancy = []

mental_health_keywords = ["depression", "mental illness", "unalive", "social anxiety", "loneliness", "stress", "lonely", "isolation", "suicide",
                            "abuse", "death", "post traumatic stress disorder", "no motivation", "therapy", "trauma", "counselling", 
                            "mood disorder", "mood swings", "mental health", "angst", "emotion", "phobia", "addiction", "stigma", "self-harm",
                            "neurosis", "abuse", "disorder", "dependence", "socialize", "help", "dead", "melancholia", "dysthemia", "tired", 
                            "trapped", "paranoia", "overwhelmed", "irritable", "bipolar", "psychologist", "well-being", "imh", "sos", 
                            "counsellor", "toxic", "insominia", "drugs", "fight", "self-esteem"]
pandemic_keywords = ["covid-19", "pandemic", "restriction", "social distancing", "circuit breaker", "pre-covid", "coronavirus", "quarantine", 
                        "lockdown", "wfh", "tighter measures", "phase 2", "vaccine", "community cases", "epidemic", "mask", "asymptomatic",
                        "fatality rate", "clinical trial", "contact tracing", "contactless", "national emergency", "transmission", "screening", 
                        "testing", "hospital", "healthcare worker", "death", "symptomatic", "symptoms", "isolation", "safe distancing",
                        "physical distancing", "outbreak", "cluster", "spread", "hygiene", "crisis", "new normal", "uncertainty", "zoom", 
                        "retrenchment", "variant", "heightened alert", "stay home notice", "shn", "hbl", "safe entry", "home based learning",
                        "frontline", "school closure", "job insecurity"]
for rowIndex, row in output_data.iterrows():
    content = row["selftext"].lower()
    mh = any(word in content for word in mental_health_keywords)
    contain_mental_health.append(mh)
    covid = any(word in content for word in pandemic_keywords)
    contain_covid.append(covid)
    contain_sg.append(True)

    if mh and covid and True:
        relevancy.append(True)
    else:
        relevancy.append(False)
output_data["contain_sg"] = contain_sg
output_data["contain_covid"] = contain_covid
output_data["contain_mental_health"] = contain_mental_health
output_data["Relevancy"] = relevancy
output_data

C:\Users\20jam\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,created_utc,author,score,num_comments,clean_selftext,selftext,year,month,contain_sg,contain_covid,contain_mental_health,Relevancy
0,2021-05-19,summerfellxx,8,6,know mostly test child really need know wrong ...,I know mostly these tests are for children. Bu...,2021,05,True,False,True,False
1,2021-03-29,ElijahThor00,5,6,people say half war report mental illness long...,People say half the war is won when you report...,2021,03,True,False,True,False
2,2021-01-19,kanicroquette,42,9,might eating disorder know want self diagnose ...,"I might have an eating disorder, I don’t know ...",2021,01,True,False,True,False
3,2021-01-18,Vivid-Wing-3460,44,7,ofs long period time sit really reflect unsolv...,After o's I have this long period of time to s...,2021,01,True,False,True,False
4,2021-01-13,chin_chin_daisuki,21,8,facing severe adjustment problem mental breakd...,"Facing severe adjustment problems, had a menta...",2021,01,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
12605,2020-04-25,QualitativeEconomy,2,0,tbh kept vaccine development news want generat...,Tbh I havent kept up with the vaccine developm...,2020,04,True,True,False,False
12606,2019-03-13,wintersoju,7,8,different brand vaccine available difference g...,Are there different brands of the vaccine avai...,2019,03,True,True,False,False
12607,2019-02-10,yummydubu,1,20,currently researching vaccination wondering ma...,Currently researching the vaccination and wond...,2019,02,True,True,False,False
12608,2019-10-29,jasonrodriguez_DT,0,6,checked couple site including ministry health ...,I checked a couple of sites including your Min...,2019,10,True,True,False,False


In [5]:
# filter out rows with null entries
output_data = output_data[output_data["clean_selftext"].notnull()]

In [6]:
# sentiment score, sentiment_class columns
import numpy as np
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
sentiment = SentimentIntensityAnalyzer()

output_data['sentiment score'] = output_data["clean_selftext"].apply(lambda x: sentiment.polarity_scores(x))
output_data["sentiment score"] = output_data["sentiment score"].apply(pd.Series)["compound"]

def get_sentiment_labels(value):
    if value <= 1.0 and value >= 0.6:
        return "Very Positive"
    elif value < 0.6 and value >= 0.1:
        return "Positive"
    elif value < 0.1 and value > -0.10:
        return "Neutral"
    elif value <= 0.1 and value >-0.55:
        return "Negative"
    else:
        return "Very Negative"
output_data["sentiment_class"] = output_data['sentiment score'].apply(get_sentiment_labels)
output_data.head()

[nltk_data] Error loading vader_lexicon: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
C:\Users\20jam\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,created_utc,author,score,num_comments,clean_selftext,selftext,year,month,contain_sg,contain_covid,contain_mental_health,Relevancy,sentiment score,sentiment_class
0,2021-05-19,summerfellxx,8,6,know mostly test child really need know wrong ...,I know mostly these tests are for children. Bu...,2021,05,True,False,True,False,-0.7310,Very Negative
1,2021-03-29,ElijahThor00,5,6,people say half war report mental illness long...,People say half the war is won when you report...,2021,03,True,False,True,False,-0.5606,Very Negative
2,2021-01-19,kanicroquette,42,9,might eating disorder know want self diagnose ...,"I might have an eating disorder, I don’t know ...",2021,01,True,False,True,False,-0.4767,Negative
3,2021-01-18,Vivid-Wing-3460,44,7,ofs long period time sit really reflect unsolv...,After o's I have this long period of time to s...,2021,01,True,False,True,False,-0.9817,Very Negative
4,2021-01-13,chin_chin_daisuki,21,8,facing severe adjustment problem mental breakd...,"Facing severe adjustment problems, had a menta...",2021,01,True,False,False,False,-0.8126,Very Negative


In [7]:
# save output_data as csv
output_data.to_csv("C:\\Users\\20jam\\Documents\\a teabag of joy\\nus\\projects so far\\my-code-OMDENA\\custom_data_for_risk_model.csv")